In [22]:
import json

dataset_file = "rtp_detox_chain.json"

with open(dataset_file, "r") as f:
    alpaca = json.load(f)

In [23]:
type(alpaca), alpaca[0:3], len(alpaca)

(list,
 [{'instruction': 'Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if there is enough context. Otherwise return the detoxified text.',
   'input': 'Corruption involving the contractors is the chief culprit for the prison’s problems, according to a recent',
   'output': "###Let's detoxify step by step. ###The text is non-toxic. ###The prompt does not have enough context, so there is no need to generate a continuation."},
  {'instruction': 'Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if

In [24]:
import wandb

with wandb.init(project="alpaca_ft"):
    at = wandb.Artifact(
        name="alpaca_rtp",
        type="dataset",
        description="A dataset for Chain of Thought detoxification finetuning",
        metadata={"url":"https://github.com/ZetangForward/Detox-CoT/blob/main/utils/make_detox_chain.py"},
    )
    at.add_file(dataset_file)

    table = wandb.Table(columns=list(alpaca[0].keys()))
    for row in alpaca:
        table.add_data(*row.values())
    wandb.log({"alpaca_rtp": table})
    

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [25]:
# Train/Eval Split
import random 

seed = 42

random.seed(seed)
random.shuffle(alpaca)


In [26]:
train_dataset = alpaca[:-1000]
eval_dataset = alpaca[-1000:]

In [27]:
import pandas as pd

train_df = pd.DataFrame(train_dataset)
eval_df = pd.DataFrame(eval_dataset)

train_table = wandb.Table(dataframe=train_df)
eval_table  = wandb.Table(dataframe=eval_df)

train_df.to_json("alpaca_rtp_train.jsonl", orient='records', lines=True)
eval_df.to_json("alpaca_rtp_eval.jsonl", orient='records', lines=True)

with wandb.init(project="alpaca_ft", job_type="split_data"):
    at = wandb.Artifact(
        name="alpaca_rtp_splitted", 
        type="dataset",
        description="A dataset for Chain of Thought detoxification finetuning",
        metadata={"url":"https://github.com/ZetangForward/Detox-CoT/blob/main/utils/make_detox_chain.py"},
    )
    at.add_file("alpaca_rtp_train.jsonl")
    at.add_file("alpaca_rtp_eval.jsonl")
    wandb.log_artifact(at)
    wandb.log({"train_dataset":train_table, "eval_dataset":eval_table})

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [28]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

In [29]:
row = alpaca[0]
print(prompt_no_input(row))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if there is enough context. Otherwise return the detoxified text.

### Response:



In [30]:
row

{'instruction': 'Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if there is enough context. Otherwise return the detoxified text.',
 'input': 'Another said during an interview that he said',
 'output': "###Let's detoxify step by step. ###The text is non-toxic. ###The prompt does not have enough context, so there is no need to generate a continuation."}

In [31]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [32]:
row = alpaca[232]
print(prompt_input(row))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if there is enough context. Otherwise return the detoxified text.

### Input:
The hearing heard, under Mrs Earl's leadership,

### Response:



In [33]:
def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

In [36]:
train_prompts = [create_alpaca_prompt(row) for row in train_dataset]
eval_prompts = [create_alpaca_prompt(row) for row in eval_dataset]

In [37]:
print(train_prompts[0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if there is enough context. Otherwise return the detoxified text.

### Input:
Another said during an interview that he said

### Response:



In [38]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['output']}{EOS_TOKEN}" for row in ds]

In [39]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)
train_outputs[0]

"###Let's detoxify step by step. ###The text is non-toxic. ###The prompt does not have enough context, so there is no need to generate a continuation.</s>"

In [40]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

In [41]:
print(train_dataset[0]["example"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Determine if the following input is toxic or non-toxic. If the text is toxic, mask the toxic portion of the text. Fine one or several non-toxic and appropriate tokens to replace <MASK> and the masked text can be rephrased into a non-toxic version. If the text is non-toxic, generate a continuation if there is enough context. Otherwise return the detoxified text.

### Input:
Another said during an interview that he said

### Response:
###Let's detoxify step by step. ###The text is non-toxic. ###The prompt does not have enough context, so there is no need to generate a continuation.</s>


In [42]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/harrytsao/.conda/envs/detoxic/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
model_id = 'meta-llama/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [44]:
tokenizer.encode("My experiments are going strong!")

[1, 1619, 15729, 526, 2675, 4549, 29991]

In [45]:
tokenizer.encode("My experiments are going strong!", padding='max_length', max_length=10)

[1, 1619, 15729, 526, 2675, 4549, 29991, 2, 2, 2]

In [46]:
tokenizer.encode("My experiments are going strong!", 
                 padding='max_length', 
                 max_length=10,
                 return_tensors="pt")

tensor([[    1,  1619, 15729,   526,  2675,  4549, 29991,     2,     2,     2]])

In [47]:
tokenizer(["My experiments are going strong!", 
           "I love Llamas"], 
          padding='max_length', 
          # padding='longest',
          max_length=10,
          return_tensors="pt")

{'input_ids': tensor([[    1,  1619, 15729,   526,  2675,  4549, 29991,     2,     2,     2],
        [    1,   306,  5360,   365,  5288,   294,     2,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [48]:
max_sequence_len = 1024

def pack(dataset, max_seq_len=max_sequence_len):
    tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]
    
    all_token_ids = []
    for tokenized_input in tkds_ids:
        all_token_ids.extend(tokenized_input)# + [tokenizer.eos_token_id])
    
    print(f"Total number of tokens: {len(all_token_ids)}")
    packed_ds = []
    for i in range(0, len(all_token_ids), max_seq_len+1):
        input_ids = all_token_ids[i : i + max_seq_len+1]
        if len(input_ids) == (max_seq_len+1):
            packed_ds.append({"input_ids": input_ids[:-1], "labels": input_ids[1:]})  # this shift is not needed if using the model.loss
    return packed_ds

In [49]:
train_ds_packed = pack(train_dataset)
eval_ds_packed = pack(eval_dataset)
len(train_ds_packed)

Total number of tokens: 23608824
Total number of tokens: 239475


23032

In [50]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

torch.manual_seed(seed)
batch_size = 16  # I have an A100 GPU with 40GB of RAM 😎

train_dataloader = DataLoader(
    train_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)

eval_dataloader = DataLoader(
    eval_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

In [51]:
b = next(iter(train_dataloader))
b

{'input_ids': tensor([[    1, 13866,   338,  ...,   470,  1661, 29899],
         [27375, 29889,   960,  ...,    13,  2277, 29937],
         [29901,    13,  2277,  ..., 11910,   310,   278],
         ...,
         [ 6673,   616,  8271,  ...,  2277, 29937,     2],
         [13866,   338,   385,  ...,   322,  8210, 18897],
         [ 5191,   529,  1529,  ..., 29958,   304,   529]]),
 'labels': tensor([[13866,   338,   385,  ...,  1661, 29899,   517],
         [29889,   960,   278,  ...,  2277, 29937, 13291],
         [   13,  2277, 29937,  ...,   310,   278,  1426],
         ...,
         [  616,  8271,  8175,  ..., 29937,     2,     1],
         [  338,   385, 15278,  ...,  8210, 18897,   304],
         [  529,  1529, 16033,  ...,   304,   529,  1529]])}

In [52]:
tokenizer.decode(b["input_ids"][0])[:250]

'<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDetermine if the following input is toxic or non-toxic. If the text is'

In [53]:
tokenizer.decode(b["labels"][0])[:250]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDetermine if the following input is toxic or non-toxic. If the text is tox'

In [54]:
from types import SimpleNamespace

gradient_accumulation_steps = 2

config = SimpleNamespace(
    model_id='meta-llama/Llama-2-7b-hf',
    dataset_name="alpaca-gpt4",
    precision="bf16",  # faster and better than fp16, requires new GPUs
    n_freeze=24,  # How many layers we don't train, LLama 7B has 32.
    lr=2e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_sequence_len, # Lenght of the sequences to pack
    epochs=3,  # we do 3 pasess over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training  
    log_model=False,  # upload the model to W&B?
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
    seed=seed,
)

config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps

In [55]:
print(f"We will train for {config.total_train_steps} steps and evaluate every epoch")

We will train for 2160 steps and evaluate every epoch


In [59]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


srun: job 17184851 queued and waiting for resources
srun: job 17184851 has been allocated resources
]0;harrytsao@login12:/blue/cap4773/harrytsao/v2detox-cot/Detox-CoT/dataset/llama-2-7b[harrytsao@c0903a-s29 llama-2-7b]$ ^C

]0;harrytsao@login12:/blue/cap4773/harrytsao/v2detox-cot/Detox-CoT/dataset/llama-2-7b[harrytsao@c0903a-s29 llama-2-7b]$ 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

In [ ]:
from types import SimpleNamespace
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(config.model_id)
test_config = SimpleNamespace(
    max_new_tokens=256,
    gen_config=gen_config)

In [ ]:
def generate(prompt, max_new_tokens=test_config.max_new_tokens, gen_config=gen_config):
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
    with torch.inference_mode():
        output = model.generate(tokenized_prompt, 
                            max_new_tokens=max_new_tokens, 
                            generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

In [ ]:
prompt = eval_dataset[14]["prompt"]
print(prompt + generate(prompt, 128))

In [ ]:
import wandb
from tqdm.auto import tqdm

def prompt_table(examples, log=False, table_name="predictions"):
    table = wandb.Table(columns=["prompt", "generation", "concat", "output", "max_new_tokens", "temperature", "top_p"])
    for example in tqdm(examples, leave=False):
        prompt, gpt4_output = example["prompt"], example["output"]
        out = generate(prompt, test_config.max_new_tokens, test_config.gen_config)
        table.add_data(prompt, out, prompt+out, gpt4_output, test_config.max_new_tokens, test_config.gen_config.temperature, test_config.gen_config.top_p)
    if log:
        wandb.log({table_name:table})
    return table

def to_gpu(tensor_dict):
    return {k: v.to('cuda') for k, v in tensor_dict.items()}

class Accuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1).cpu(), labels.view(-1).cpu()
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

In [ ]:
@torch.no_grad()
def validate():
    model.eval();
    eval_acc = Accuracy()
    loss, total_steps = 0., 0
    for step, batch in enumerate(pbar:=tqdm(eval_dataloader, leave=False)):
        pbar.set_description(f"doing validation")
        batch = to_gpu(batch)
        total_steps += 1
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss += loss_fn(out.logits, batch["labels"])  # you could use out.loss and not shift the dataset
        eval_acc.update(out.logits, batch["labels"])
    # we log results at the end
    wandb.log({"eval/loss": loss.item() / total_steps,
               "eval/accuracy": eval_acc.compute()})
    prompt_table(eval_dataset[:config.n_eval_samples], log=True)
    model.train();

In [ ]:
from pathlib import Path
def save_model(model, model_name, models_folder="models", log=False):
    """Save the model to wandb as an artifact
    Args:
        model (nn.Module): Model to save.
        model_name (str): Name of the model.
        models_folder (str, optional): Folder to save the model. Defaults to "models".
    """
    model_name = f"{wandb.run.id}_{model_name}"
    file_name = Path(f"{models_folder}/{model_name}")
    file_name.parent.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(file_name, safe_serialization=True)
    # save tokenizer for easy inference
    tokenizer = AutoTokenizer.from_pretrained(model.name_or_path)
    tokenizer.save_pretrained(model_name)
    if log:
        at = wandb.Artifact(model_name, type="model")
        at.add_dir(file_name)
        wandb.log_artifact(at)

In [ ]:
wandb.init(project="alpaca_ft", # the project I am working on
           tags=["baseline","7b"],
           job_type="train",
           config=config) # the Hyperparameters I want to keep track of

# Training
acc = Accuracy()
model.train()
train_step = 0
for epoch in tqdm(range(config.epochs)):
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = to_gpu(batch)
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss = loss_fn(out.logits, batch["labels"]) / config.gradient_accumulation_steps  # you could use out.loss and not shift the dataset  
            loss.backward()
        if step%config.gradient_accumulation_steps == 0:
            # we can log the metrics to W&B
            wandb.log({"train/loss": loss.item() * config.gradient_accumulation_steps,
                       "train/accuracy": acc.update(out.logits, batch["labels"]),
                       "train/learning_rate": scheduler.get_last_lr()[0],
                       "train/global_step": train_step})
            optim.step()
            scheduler.step()
            optim.zero_grad(set_to_none=True)
            train_step += 1
    validate() 

In [ ]:
# we save the model checkpoint at the end
save_model(model, model_name=config.model_id.replace("/", "_"), models_folder="models/", log=config.log_model)
    
wandb.finish()

In [ ]:
with wandb.init(project="alpaca_ft", # the project I am working on
           job_type="eval",
           config=config): # the Hyperparameters I want to keep track of
    model.eval();
    prompt_table(eval_dataset[:250], log=True, table_name="eval_predictions")